## **Aplicación**

Aplicación web interactiva desarrollada en Streamlit que utiliza un modelo de Máquina de Soporte Vectorial (SVM) previamente entrenado y optimizado para clasificar automáticamente el riesgo de diabetes en pacientes. El sistema procesa archivos CSV con datos clínicos y devuelve el mismo archivo enriquecido con etiquetas de riesgo (Bajo/Alto), priorizando la máxima sensibilidad para no perder casos positivos.

### **Modelo**

El sistema emplea un clasificador SVM con kernel optimizado previamente mediante validación cruzada. El modelo fue seleccionado por su robustez en espacios de dimensionalidad media y su capacidad de generalización con datasets clínicos de tamaño moderado (n=2,533). Se priorizó la métrica de recall (sensibilidad: 95.7%) para minimizar falsos negativos en el screening diabético.


### **Variables de Entrada Requeridas**


| Variable    | Tipo       | Descripción                 | Rango Esperado |
| ----------- | ---------- | --------------------------- | -------------- |
| `sexo`      | Categórico | Sexo biológico del paciente | F/M            |
| `edad`      | Numérico   | Edad en años                | 18-90          |
| `imc`       | Numérico   | Índice de Masa Corporal     | 15-50          |
| `glu_suero` | Numérico   | Glucosa en suero (mg/dL)    | 70-300         |
| `trig`      | Numérico   | Triglicéridos (mg/dL)       | 50-500         |
| `col_hdl`   | Numérico   | Colesterol HDL (mg/dL)      | 20-100         |
| `insulina`  | Numérico   | Insulina en ayunas (µIU/mL) | 2-50           |
| `ac_urico`  | Numérico   | Ácido úrico (mg/dL)         | 2-12           |


### **Salida**

| Columna Nueva     | Descripción               | Ejemplo                       |
| ----------------- | ------------------------- | ----------------------------- |
| `riesgo_predicho` | Clasificación binaria SVM | "Alto Riesgo" / "Bajo Riesgo" |


In [ ]:
import streamlit as st
import pandas as pd
import joblib

# ------------------------------
# Configuración de la app
# ------------------------------
st.set_page_config(
    page_title="Detección de Riesgo de Diabetes",
    layout="centered"
)

st.title("🩺 Detección de Riesgo de Diabetes")
st.write(
    "Aplicación clínica para la predicción automática del riesgo de diabetes "
    "a partir de variables metabólicas."
)

# ------------------------------
# Cargar modelo
# ------------------------------
@st.cache_resource
def cargar_modelo():
    return joblib.load("modelo_svc.pkl")

modelo = cargar_modelo()

# ------------------------------
# Subida de archivo
# ------------------------------
archivo = st.file_uploader(
    "📂 Cargue el archivo CSV del paciente",
    type=["csv"]
)

if archivo is not None:
    df = pd.read_csv(archivo)

    st.subheader("📊 Vista previa de los datos")
    st.dataframe(df.head())

    # ------------------------------
    # Verificación de columnas
    # ------------------------------
    columnas_esperadas = [
        'sexo', 'edad', 'imc', 'glu_suero',
        'trig', 'col_hdl', 'insulina', 'ac_urico'
    ]

    if not all(col in df.columns for col in columnas_esperadas):
        st.error(
            "❌ El archivo no contiene todas las columnas requeridas.\n\n"
            f"Columnas esperadas:\n{columnas_esperadas}"
        )
    else:
        # ------------------------------
        # Predicción
        # ------------------------------
        st.subheader("🔮 Predicción del modelo")

        predicciones = modelo.predict(df[columnas_esperadas])

        df_resultado = df.copy()
        df_resultado["riesgo_diabetes"] = predicciones
        df_resultado["riesgo_diabetes"] = df_resultado["riesgo_diabetes"].map({
            0: "Riesgo bajo",
            1: "Riesgo alto"
        })

        st.success("✅ Predicciones generadas correctamente")

        st.dataframe(df_resultado.head())

        # ------------------------------
        # Descargar resultados
        # ------------------------------
        csv_resultado = df_resultado.to_csv(index=False).encode("utf-8")

        st.download_button(
            label="⬇️ Descargar archivo etiquetado",
            data=csv_resultado,
            file_name="predicciones_riesgo_diabetes.csv",
            mime="text/csv"
        )

# ------------------------------
# Footer
# ------------------------------
st.markdown("---")
st.caption(
    "⚠️ Esta herramienta es de apoyo clínico y no sustituye el juicio médico."
)
